# Introduction
As part of the overall goal of the PERCEIVE project to proactively identify upcoming cybersecurity threats, social network analysis is being used to analyze relevant email lists. The subject of this notebook, the [Full Disclosure (FD) mailing list](http://seclists.org/fulldisclosure/) is a "public, vendor-neutral forum for detailed discussion of vulnerabilities and exploitation techniques, as well as tools, papers, news, and events of interest to the community."
## Problem Statement
Although email networks provide a corpus for analysis and predictive modeling, the actual FD network includes a large amount of content that is irrelevant to the project's purpose. This content includes conference invitations[[1](http://seclists.org/fulldisclosure/2017/Feb/6)], vendor announcements[[2](http://seclists.org/fulldisclosure/2016/Dec/48)], and extensive conversations on security topics[[3](http://seclists.org/fulldisclosure/2004/Jul/1026)]. The list has also included a significant amount of trolling[[4](http://seclists.org/fulldisclosure/2008/Apr/96)] and nonsense[[5](http://seclists.org/fulldisclosure/2009/Jul/289)] [[6](http://seclists.org/fulldisclosure/2004/Jul/796)].

Social network analysis will allow for the isolation of relevant discussion topics in order to develop a more accurate sample set for continuing study.
## Background
Earlier in this project, the FD email lists were developed into networks of edges and nodes, divided by year. The original csv files are available [online](https://mega.nz/#F!CUEByR5I!GY56GzTpYz68IlTqj4aQNQ!fR8jFLxL). The networks were imported into [Gephi](https://gephi.org/) to create a series of [visualization graphics](
https://mega.nz/#F!btdgFBID!hktkVrhZB5etOBBVLrgTrA).

Gephi is useful for visualization, but problematic for analysis. In particular, many of the core functions[[7](https://github.com/jaroslav-kuchar/Social-Network-Analysis/issues/2)] and plugins[[8](https://github.com/gephi/gephi/issues/1481)] once used for social network analysis are not compatible with current (0.9) software versions. Gephi specifications note that the 0.9.0 version (December 2015) "Removed [the] Clustering module"[[9](https://github.com/gephi/gephi/releases/tag/v0.9.0)] that these plugins relied upon.
## Graph overview
In the FD graphs, nodes represent either documents (i.e., emails) or authors. Edges are directed, representing authorship and replies; edge weight indicates an increasing number of replies. In the Gephi graphics, blue nodes represent authors and red nodes represent documents. 
# Social network analysis in python-igraph
## Importing graphs into igraph
[Python-igraph](http://igraph.org/python/) (igraph hereafter) allows a user to import graphs in a variety of file formats, several of which match Gephi's export options. Brief tests demonstrated that the [GraphML](http://graphml.graphdrawing.org/) format was most successful for a workflow of transferring networks from Gephi to igraph. (Attempt to import [GML](https://en.wikipedia.org/wiki/Graph_Modelling_Language) and [Pajek](http://mrvar.fdv.uni-lj.si/pajek/) caused one instance of Python to hang; further investigation would be required to identify a cause.)

In [1]:
# GraphML files may be imported into igraph with a single command (`Read_GraphML`) and then analyzed:

from igraph import *

# import the GraphML files previously exported from Gephi
fd2016 = Graph.Read_GraphML('graphml/2016.graphml')


## Identifying graph features in igraph

In [2]:
from igraph import *

# import the GraphML files previously exported from Gephi
fd2016 = Graph.Read_GraphML('graphml/2016.graphml')

fd2016['name'] = 'Full Disclosure network 2016'

# summary reports the edges, nodes, and overall attributes in use. 
summary(fd2016)

IGRAPH D-W- 984 758 -- Full Disclosure network 2016
+ attr: name (g), Color (v), Label1 (v), Label2 (v), b (v), g (v), id (v), label (v), nodeType (v), r (v), size (v), x (v), y (v), Edge Label (e), id (e), weight (e)


The summary command providing the output seen above is explained in the igraph documentation[[10](http://igraph.org/python/doc/igraph.summary'.GraphSummary-class.html)]. In this example, the four-character code "D-W-" indicates that the graph is directed and weighted. The graph has 984 vertices (nodes) and 758 edges. 

The attributes given in the summary ("name", "Color", etc.) are those for the graph (g), vertices (v), or edges (e).



In [3]:
from igraph import *

# import the GraphML files previously exported from Gephi
fd2016 = Graph.Read_GraphML('graphml/2016.graphml')
fd2006 = Graph.Read_GraphML('graphml/2006.graphml')

# show the id attribute of the first ten vertices
print('ID list of first ten vertices in network (2016):')
print(fd2016.vs[0:10]['id'])

# identify vertices with the highest degree and betweenness
# (This sample is borrowed directly from the tutorial!)

print('\nIDs of vertices with highest degree (2016):',
      fd2016.vs.select(_degree = fd2016.maxdegree())['id'])
print('\nIDs of vertices with highest degree (2006):',
      fd2006.vs.select(_degree = fd2006.maxdegree())['id'])


ID list of first ten vertices in network (2016):
['U?ur Cihan KO� <u.cihan.koc () gmail com>', 'Stefan Kanthak <stefan.kanthak () nexgo de>', 'Eitan Caspi <eitanc () yahoo com>', 'MustLive <mustlive () websecurity com ua>', 'vishnu raju <rajuvishnu52 () gmail com>', 'Sebastian Perez <s3bap3 () gmail com>', 'Pierre Kim <pierre.kim.sec () gmail com>', 'freeman <freeman () hackerzvoice net>', 'Ahmed Sultan <0x4148 () gmail com>', 'CSW Research Lab <disclose () cybersecurityworks com>']

IDs of vertices with highest degree (2016): ['Vulnerability Lab <research () vulnerability-lab com>']

IDs of vertices with highest degree (2006): ['Gadi Evron <ge () linuxbox org>']


## Plotting graphics in igraph
The GraphML files from Gephi include the ids as _labels_, and so they are included by default when igraph creates visualizations. These may be eliminated by deleting the attribute.

Igraph handles visualization through _layouts_. Layouts are separate objects from the graphs themselves; multiple layouts can be created per graph.

Visual graphs are created via `plot` commands. 

In [4]:
from igraph import *

# import the GraphML files previously exported from Gephi
fd2016 = Graph.Read_GraphML('graphml/2016.graphml')

# delete label attributes to avoid visual clutter. In this example, the 'label'
# attribute is a duplicate of the 'ID' attribute so there is no need to save the label
del fd2016.vs['label']

layout = fd2016.layout('fr') # create a Fruchterman Reingold layout for the graph

png = plot(fd2016, 'output/2016-fr.png', layout=layout, bbox=(1000,1000), margin=100,
           edge_curved=True)  # the bbox keyword argument defines the dimensions



The resulting graphic provides a visual overview of the FD 2016 network:

![An igraph-generated FD graph for 2016](output/2016-fr.png)

